## To Merge All Kinds of Bhavcopy

In [ ]:
import os
import pandas as pd
from datetime import datetime

def get_file_path(exchange_type, date_str_ddmmyyyy):
    # Parse the date input
    date_obj = datetime.strptime(date_str_ddmmyyyy, "%d%m%Y")
    date_str_file = date_obj.strftime("%Y%m%d")
    month_folder_abbr = date_obj.strftime("%b_%y").upper()   # e.g., JUN_25
    month_folder_full = date_obj.strftime("%B_%y").upper()   # e.g., JUNE_25
    month_folder_title = date_obj.strftime("%B")             # e.g., June

    base_path = r"E:\Back-Up Files\Data Team\BSE - NSE"

    if exchange_type == "BSE_FO":
        folder = os.path.join(base_path, f"Bhavcopy 2025\\{month_folder_full}\\BSE")
        filename = f"BhavCopy_BSE_FO_0_0_0_{date_str_file}_F_0000.csv"

    elif exchange_type == "NSE_FO":
        folder = os.path.join(base_path, f"Bhavcopy 2025\\{month_folder_full}\\NSE")
        filename = f"BhavCopy_NSE_FO_0_0_0_{date_str_file}_F_0000.csv"

    elif exchange_type == "NSE_CM":
        folder = os.path.join(base_path, f"Bhavcopy 2025\\CM_BHAVCOPY_25\\{month_folder_abbr}\\NSE_CM")
        filename = f"BhavCopy_NSE_CM_0_0_0_{date_str_file}_F_0000.csv"

    elif exchange_type == "BSE_CM":
        folder = os.path.join(base_path, f"Bhavcopy 2025\\CM_BHAVCOPY_25\\{month_folder_abbr}\\BSE_CM")
        filename = f"BhavCopy_BSE_CM_0_0_0_{date_str_file}_F_0000.csv"

    elif exchange_type == "GT_FO":
        folder = os.path.join(base_path, f"GT-Bhavcopy 2025\\{month_folder_title}")
    # Change date format to YYMMDD for GT files:
        date_str_yt = date_obj.strftime("%y%m%d")  # e.g. 250625
        filename = f"G_T_Bhavcopy_FO_{date_str_yt}.csv"


    elif exchange_type == "CN_FO":
        folder = os.path.join(base_path, f"CN-Bhavcopy 2025\\{month_folder_title}")
        filename = f"F_CN01_NSE_{date_str_ddmmyyyy}.csv"  # <--- Use DDMMYYYY here!

    elif exchange_type == "MS_FO":
        folder = os.path.join(base_path, f"MS-Bhavcopy 2025\\{month_folder_title}")
        filename = f"MS_{date_str_file}_01_Symbol.csv"  # Adjusted filename format based on your example

    else:
        raise ValueError(f"❌ Unknown exchange type: {exchange_type}")

    return os.path.join(folder, filename)


def merge_files(date_str_ddmmyyyy, file_types=None):
    if file_types is None:
        file_types = ["BSE_FO", "NSE_FO", "NSE_CM", "BSE_CM", "GT_FO", "CN_FO", "MS_FO"]

    merged_df = pd.DataFrame()
    expected_columns = None
    files_found = 0

    for ftype in file_types:
        print(f"\n🔍 Looking for: {ftype}")

        try:
            file_path = get_file_path(ftype, date_str_ddmmyyyy)

            folder = os.path.dirname(file_path)
            if not os.path.exists(folder):
                print(f"❌ Folder not found: {folder}")
                continue

            if not os.path.exists(file_path):
                print(f"❌ File not found for {ftype} at {file_path}")
                continue

            df = pd.read_csv(file_path)
            df["Source"] = ftype

            if expected_columns is None:
                expected_columns = df.columns
                merged_df = df
            else:
                # Union of columns
                all_columns = expected_columns.union(df.columns)
                merged_df = merged_df.reindex(columns=all_columns)
                df = df.reindex(columns=all_columns)
                merged_df = pd.concat([merged_df, df], ignore_index=True)
                expected_columns = all_columns

            files_found += 1
            print(f"✅ Added: {file_path}")

        except Exception as e:
            print(f"❌ Error processing {ftype}: {e}")

    output_folder = r"E:\Back-Up Files\Data Team\BSE - NSE\Merged_Bhavcopy_2025\All Files"
    os.makedirs(output_folder, exist_ok=True)
    output_file = os.path.join(output_folder, f"merged_bhavcopy_{date_str_ddmmyyyy}.csv")

    if files_found > 0 and not merged_df.empty:
        merged_df.to_csv(output_file, index=False)
        print(f"\n✅ Merged file saved to: {output_file}")
    else:
        print("\n⚠️ No valid files found. Nothing to merge.")


if __name__ == "__main__":
    # Change this date as needed in DDMMYYYY format
    date_input = "25062025"
    merge_files(date_input)


## Import Bhavcopy 

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib

# ---------- SQL Server connection details ----------
server = 'AG-SERVER-043'
database = 'harsh_data'
username = 'data05'
password = 'sai@123'

# ---------- Create the SQLAlchemy engine ----------
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password}"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ---------- Folder path to start looking for CSV files ----------
root_folder = r"E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\FEB_2025_CM - Copy"

# ---------- Loop through folders and process CSV files ----------
for dirpath, _, filenames in os.walk(root_folder):
    for file in filenames:
        if file.lower().endswith('.csv'):
            file_path = os.path.join(dirpath, file)
            table_name = os.path.splitext(file)[0].replace(' ', '_').replace('-', '_')

            print(f"\n📥 Importing: {file_path} → Table: {table_name}")

            try:
                # Read the CSV file
                df = pd.read_csv(file_path, low_memory=False)


                # Upload to SQL Server (replace table if it exists)
                df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
                print(f"✅ Successfully imported into table: {table_name}")
            except Exception as e:
                print(f"❌ Failed to import {file}: {e}")
